**Import needed packages/modules**

In [ ]:
# Cell 1
import matplotlib.pyplot as plt
import numpy as np
from numba import int64, float64, vectorize

**Declare a numba accelerated function that computes the Halton QRNG**
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
# Cell 2
@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h

**Set the number of random $dots$ (samples) to take**

In [ ]:
# Cell 3
total_dots = 40_600

**Take $n$ "random" samples of 2D Cartesian points $(x,y)$ using the Halton sequence**
1. Scale the results so $-2\le x_{rng}\leq 2$ and $0\le y_{rng}\leq 5$
2. The sample area is thus $(-2...2)\times(0..5)=20$


In [ ]:
# Cell 4
x = (1 - halton(np.arange(total_dots), 2)) * 4 - 2
y = (1 - halton(np.arange(total_dots), 3)) * 5
print(x)
print(y)

**Create an array $d$ containing $y_{rnd}-f(x_{rnd})$**


In [ ]:
# Cell 5
d = y - (4 - x**2)
print(d)

**Create arrays of $(x,y)$ coordinates that are "above" or "on or below" the parabola**\
Here $f(x)=4-x^2$ so if $d>0$ then the sample point is "above" the curve

In [ ]:
# Cell 6
x_in = x[d <= 0.0]
y_in = y[d <= 0.0]
x_out = x[d > 0.0]
y_out = y[d > 0.0]

**Calculate the absolute percent error in the area estimation**
1. The actual/expected definite integral is $\frac{32}{3}=10.6666...$
2. The observed/estimated area using the Monte Carlo formulation $\large=20\times\frac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 7
act = 32 / 3
est = 20 * np.count_nonzero(d <= 0.0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")

**Display the scatter plot of the Monte Carlo estimation**

In [ ]:
# Cell 8
plt.figure(figsize=(8, 6))
plt.scatter(x_in, y_in, color="red", marker=".", s=0.5)
plt.scatter(x_out, y_out, color="blue", marker=".", s=0.5)
plt.title("$y=-x^2+4$")
plt.xlabel("x")
plt.ylabel("y")
plt.show()